In [3]:
import requests
from lxml import html
import json

In [19]:
link = 'https://www.hse.ru/ba/ling/students/diplomas/page{}.html'

page = requests.get(link.format(1))  

In [5]:
print(page.text[:251])

<!DOCTYPE html>
<!-- (c) Art. Lebedev Studio | http://www.artlebedev.com/ -->
<html xmlns:perl="urn:perl" lang="ru"><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta nam


In [35]:
tree = html.fromstring(page.content)

In [42]:
profs = tree.xpath('//div[@data-title="Руководитель" and not(a)]/text()|//div[@data-title="Руководитель"]/a/text()')
print(profs)    


['Бонч-Осмоловская Анастасия Александровна', 'Ляшевская Ольга Николаевна', 'Акинина Юлия Сергеевна', 'Толдова Светлана Юрьевна', 'Ландер Юрий Александрович', 'Плунгян Владимир Александрович\n\n\t\t\t\t\t\t\xa0\n\t\t\t\t\t', 'Резникова Татьяна Исидоровна', 'Ляшевская Ольга Николаевна', 'Орехов Борис Валерьевич', 'Князев Сергей Владимирович', 'Летучий Александр Борисович', 'Толдова Светлана Юрьевна', 'Толдова Светлана Юрьевна', 'Толдова Светлана Юрьевна', 'Ляшевская Ольга Николаевна', 'Орехов Борис Валерьевич', 'Бонч-Осмоловская Анастасия Александровна', 'Добрушина Нина Роландовна', 'Рахилина Екатерина Владимировна', 'Выренкова Анастасия Сергеевна', 'Виноградова Ольга Ильинична', 'Шнитке Екатерина  Леонидовна', 'Ляшевская Ольга Николаевна', 'Мороз Георгий Алексеевич', 'Драгой Ольга Викторовна', 'Резникова Татьяна Исидоровна', 'Кутузов Андрей Борисович', 'Летучий Александр Борисович', 'Орехов Борис Валерьевич', 'Галямина Юлия Евгеньевна']


In [12]:
year = tree.xpath('//div[@data-title="Год защиты"]/text()')
name = tree.xpath('//div[@data-title="ФИО студента"]/text()')
title = tree.xpath('//div[@data-title="Название работы"]/a/text()')
link_title = tree.xpath('//div[@data-title="Название работы"]/a/@href')

works = zip(profs, year, name, title, link_title)
w = next(works)
print(w)

('Бонч-Осмоловская Анастасия Александровна', '2016', 'Колбасов Матвей Григорьевич', 'Автоматическая классификация нарративной структуры текстов личных дневников', '//www.hse.ru/edu/vkr/184654252')


In [14]:
page2 = requests.get('https:' + w[4])  

In [24]:
tree2 = html.fromstring(page2.content)
annotation = tree2.xpath('/html/body/div[1]/div[3]/div[2]/div[2]/div/div/div[1]/div/div/div/div[1]/div[2]/div[3]/text()')[0]
file = tree2.xpath('/html/body/div[1]/div[3]/div[2]/div[2]/div/div/div[1]/div/div/div/div[1]/div[2]/div[4]/a/@href')[0]

page3 = requests.get('https:' + w[4].replace('/edu', '/en/edu')) 
tree3 = html.fromstring(page3.content)
en_title = tree3.xpath('/html/body/div[1]/div[3]/div[2]/div[2]/div/div/div[1]/div/div/div/div[1]/div[2]/div[2]/div[2]/text()')[0].strip()
en_annotation = tree3.xpath('/html/body/div[1]/div[3]/div[2]/div[2]/div/div/div[1]/div/div/div/div[1]/div[2]/div[3]/text()')[0]
en_name = tree3.xpath('/html/body/div[1]/div[3]/div[2]/div[2]/div/div/div[1]/div/div/div/div[1]/div[2]/div[2]/div[1]/text()')[0]
en_prof = tree3.xpath('/html/body/div[1]/div[3]/div[2]/div[2]/div/div/div[1]/div/div/div/div[1]/div[2]/div[2]/div[3]/a/text()')[0]
print(annotation, file, en_title, en_annotation, sep='\n')

Задача автоматического определения временной референции события является одной из центральных задач в области информационного поиска и извлечения фактов. Это исследование посвящено решению задачи автоматического определения временной референции события на материале личных дневников. Для этого была изучена нарративная система дневниковых записей. Был собран обширный корпус текстов дневников, и разработана система классов клауз, позволяющая выделить необходимые типы клауз для дальнейшего изучения. Была создана система параметров и алгоритм разметки этих параметров, с помощью которого был размечен корпус дневниковых текстов. Были вручную собраны и размечены обучающие корпуса, необходимые для работы алгоритмов машинного обучения. С помощью последних была произведена классификация дневниковых записей, позволяющая выявить пригодные для дальнейшего извлечения из них событийных сущностей и временной референции.
http://lms.hse.ru/ap_service.php?getwork=1&guid=CD7E8659-1FBD-4983-86EE-4C3FBFA8A5C

In [27]:
def split_name(who, text, year, en_text):
    n = text.split()
    a = en_text.split()
    if who == 'a':
        return {"a_first_name": n[1], "a_second_name": n[0], "a_fathers_name": n[2], "admission_year": int(year) - 5,
               "a_first_name_en": ' '.join(a[:-1]), "a_second_name_en": a[-1]}
    else:
        return {"t_first_name": n[1], "t_second_name": n[0], "t_fathers_name": n[2],
                "t_first_name_en": ' '.join(a[:-1]), "t_second_name_en": a[-1]}
    
d = {"title": w[3], "title_en": en_title, "author": split_name('a', w[2], w[1], en_name), 
     "prof": split_name('t', w[0], w[1], en_prof),
     "keywords": "", "keywords_en": "","abstract": annotation, "abstract_en": en_annotation, 
     "form": "thesis", "course": "4", "file": file, "lang": "ru"}

print(d)

{'keywords': '', 'abstract_en': 'The task of automatic determination of the time reference of event is one of the central objects of interest in the field of information retrieval and facts extraction. This research is devoted to solving the problem of automatically determining the time reference of event on the material of personal diaries. To reach this goal narrative structure of diaries was studied. Extensive corpus of diaries texts was collected and a system of classes clause was developed, which allows to provide the necessary types of clauses for further study. A system of parameters was established and an algorithm of marking these parameters was created. With a help of this algorithm, the corpus of diaries texts was marked. Educational corpus was manually collected and marked, which was used for machine learning algorithms. The classification of the diary was produced, which allows to identify clauses suitable for the further extraction of event-related entities and temporal r

In [43]:
import requests
from lxml import html
import json

link = 'https://www.hse.ru/ba/ling/students/diplomas/page{}.html'
pages = [requests.get(link.format(num)) for num in [1,2,3,4]]

In [62]:
def split_name(who, text, year, en_text):
    n = text.split()
    a = en_text.split()
    if who == 'a':
        return {"a_first_name": n[1], "a_second_name": n[0], "a_fathers_name": n[2], "admission_year": int(year) - 4,
               "a_first_name_en": ' '.join(a[:-1]), "a_second_name_en": a[-1]}
    else:
        return {"t_first_name": n[1], "t_second_name": n[0], "t_fathers_name": n[2],
                "t_first_name_en": ' '.join(a[:-1]), "t_second_name_en": a[-1]}
    
arr = []

for page in pages:
    tree = html.fromstring(page.content)
    
    profs = tree.xpath('//div[@data-title="Руководитель" and not(a)]/text()|//div[@data-title="Руководитель"]/a/text()')
    year = tree.xpath('//div[@data-title="Год защиты"]/text()')
    name = tree.xpath('//div[@data-title="ФИО студента"]/text()')
    title = tree.xpath('//div[@data-title="Название работы"]/a/text()')
    link_title = tree.xpath('//div[@data-title="Название работы"]/a/@href')

    works = zip(profs, year, name, title, link_title)
    for w in works:
        print(w[3], '-', w[2])
        page2 = requests.get('https:' + w[4])  
        tree2 = html.fromstring(page2.content)
        
        annotation = tree2.xpath('/html/body/div[1]/div[3]/div[2]/div[2]/div/div/div[1]/div/div/div/div[1]/div[2]/div[3]/text()')[0]
        count = tree2.xpath('count(/html/body/div[1]/div[3]/div[2]/div[2]/div/div/div[1]/div/div/div/div[1]/div[2]/div)')
        if count == 4:
            file = tree2.xpath('/html/body/div[1]/div[3]/div[2]/div[2]/div/div/div[1]/div/div/div/div[1]/div[2]/div[4]/a/@href')[0]
        else:
            file = ''
        page3 = requests.get('https:' + w[4].replace('/edu', '/en/edu')) 
        tree3 = html.fromstring(page3.content)
        en_title = tree3.xpath('//div[@class="dynamic-content"]/div[2]/div[2]/text()')[0].strip()
        try:
            en_annotation = tree3.xpath('/html/body/div[1]/div[3]/div[2]/div[2]/div/div/div[1]/div/div/div/div[1]/div[2]/div[3]/text()')[0]
            en_name = tree3.xpath('//div[@class="dynamic-content"]/div[2]/div[1]/text()')[0]
            en_prof = tree3.xpath('//div[@class="dynamic-content"]/div[2]/div[3][not(a)]/text() | //div[@class="dynamic-content"]/div[2]/div[3]/a/text() ')[0]
        except:
            en_annotation, en_name, en_prof = '', w[2], w[0]
        
        d = {"title": w[3], "title_en": en_title, "author": split_name('a', w[2], w[1], en_name), 
             "prof": split_name('t', w[0], w[1], en_prof),
             "keywords": "", "keywords_en": "","abstract": annotation, "abstract_en": en_annotation, 
             "form": "thesis", "course": "4", "file": file, "lang": "ru", "year":w[1]}
        arr.append(d)
        
with open('hse_vkr.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(arr))

Автоматическая классификация нарративной структуры текстов личных дневников - Колбасов Матвей Григорьевич
Автоматическое распознавание семантических ролей с привлечением больших данных - Евстратенко Богдан Сергеевич
Адаптация и стандартизация теста APACS для оценки прагматической обработки - Грабовская Мария Андреевна
Анализ методов определения авторства на материале коротких текстов на русском языке - Присяжная Ангелина Юрьевна
Вопросительные предложения в полисинтетических языках - Касьянова Полина Алексеевна
Выражение падежных значений в урмийском новоарамейском - Саркисов Иван Владимирович
Глаголы извлечения объекта: типология прямых и переносных значений - Багдасарян Кристине Артуровна
Глагольный аспект: шкала или континуум? - Кузьменко Елизавета Алексеевна
Граф как метод описания истории русской рифмы: лингвостатистический подход и база данных - Созинова Ольга Андреевна
Дистантное взаимодействие согласных в СРЛЯ - Свешникова Варвара Сергеевна
Инструментальный аппликатив в типолог

In [52]:
import requests

f = requests.get('http://lms.hse.ru/ap_service.php?getwork=1&guid=CD7E8659-1FBD-4983-86EE-4C3FBFA8A5C3') 

In [54]:
print(f.content[:200])

b'%PDF-1.5\n%\xbf\xf7\xa2\xfe\n90 0 obj\n<< /Linearized 1 /L 711291 /H [ 911 362 ] /O 94 /E 132209 /N 40 /T 710482 >>\nendobj\n                                                                                            '


In [60]:
print(f.headers['content-disposition'].split('=')[-1])

kolbasov_mg_avtomaticheskaya-klassifikaciya-narrativnoy-struktury-tekstov-lichnyh-dnevnikov_33493.pdf


In [61]:
with open(f.headers['content-disposition'].split('=')[-1], 'wb') as pdf:
    pdf.write(f.content)